In [53]:
import pandas as pd

result1 = pd.read_csv('result csv/chatbot_responses_auto.csv')
result2 = pd.read_csv('result csv/new_chatbot_responses_auto.csv')
result3 = pd.read_csv('result csv/chatbot_responses_trained.csv')
result4 = pd.read_csv('result csv/new_chatbot_responses_trained.csv')
result5 = pd.read_csv('result csv/new_chatbot_responses_BO.csv')
result1 = result1.drop(columns=['max_similarity'])
result2

,input_question,answer,max_similarity
0,화상연고도 처방이 되나요?,저희 병원에서 사용되는 의약품은 아래층에 행복 약국에서 구매가 가능합니다.,0.713620
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,어서 오십시오. 성함과 연락처를 앞에 적어주시면 확인 후 접수 도와드리겠습니다.,0.842398
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,대기하시기 전에 접수증 작성 부탁드립니다.,0.857856
3,"피가 계속 나는데, 저 죽는 걸까요?",긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...,0.830222
4,접수 재등록해도 되나요?,마감 정산이 필요하여 지금 수납해 주시면 원활한 처리가 가능합니다.,0.777209
...,...,...,...
66,다음 예약을 안해서 예약 하려고요.,토요일 오후 여섯 시까지 진료하고 있습니다. 토요일로 예약을 도와드리겠습니다.,0.832070
67,여기서 다음 진료 예약하면 되나요?,성함과 생년월일을 말씀해 주시면 예약을 도와드리도록 하겠습니다.,0.846950
68,다음 진료 예약하려고 하는데요.,네 다음 주 수요일 오전 열시로 예약 도와드리겠습니다.,0.847497
69,다음주에 예약 시간 되는 날짜를 알려주세요.,"조회 중입니다. 예약 가능 시간 조회 결과 아홉시랑 열시 사이, 두시와 네시 사이 ...",0.844802


In [54]:
# 열 이름 변경
result1_answer = result1.rename(columns={
    'answer': 'automodel'
})
result2_answer = result2[['answer']]
result2_answer = result2_answer.rename(columns={
    'answer': 'new_automodel'
})
result3_answer = result3[['best_answer']]
result3_answer = result3_answer.rename(columns={
    'best_answer': 'trained_model'
})
result4_answer = result4[['best_answer']]
result4_answer = result4_answer.rename(columns={
    'best_answer': 'new_trained_model'
})
result5_answer = result5[['best_answer']]
result5_answer = result5_answer.rename(columns={
    'best_answer': 'BO_model'
})

In [58]:
# 두 DataFrame을 열 방향으로 합치기
df_renamed = pd.concat([result1_answer,result2_answer.reset_index(drop=True),result3_answer.reset_index(drop=True),result4_answer.reset_index(drop=True),result5_answer.reset_index(drop=True)], axis=1)
df_renamed.head()
df_renamed.to_excel('output.xlsx', index=False)

In [56]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import font_manager, rc

# 한글 글꼴 설정 (예: 맑은 고딕)
font_path = "C:/Windows/Fonts/malgun.ttf"  # 맑은 고딕의 경로 (Windows 기준)
font_manager.fontManager.addfont(font_path)
rc('font', family='Malgun Gothic')

# 글자 수에 따라 줄바꿈 처리 함수
def wrap_text(text, width=12):
    if len(text) > width:
        return '\n'.join([text[i:i+width] for i in range(0, len(text), width)])
    return text

# 모든 질문에 대해 줄바꿈 처리
df_renamed['input_question'] = df_renamed['input_question'].apply(wrap_text)
df_renamed['automodel'] = df_renamed['automodel'].apply(wrap_text)
df_renamed['new_automodel'] = df_renamed['new_automodel'].apply(wrap_text)
df_renamed['trained_model'] = df_renamed['trained_model'].apply(wrap_text)
df_renamed['new_trained_model'] = df_renamed['new_trained_model'].apply(wrap_text)
df_renamed['BO_model'] = df_renamed['BO_model'].apply(wrap_text)


# PDF 파일 저장
pdf_file_path = 'output.pdf'

# 최대 행 수 설정
max_rows_per_page = 9

# PdfPages 객체 생성
with PdfPages(pdf_file_path) as pdf:
    for start in range(0, len(df_renamed), max_rows_per_page):
        end = start + max_rows_per_page
        page_data = df_renamed[start:end]

        # A4 용지 크기로 설정 (가로: 8.27, 세로: 11.69)
        fig, ax = plt.subplots(figsize=(8.27, 14))  # A4 세로 길이 늘림
        ax.axis('tight')
        ax.axis('off')
        table_data = page_data.values
        column_labels = page_data.columns

        # 테이블 생성
        table = ax.table(cellText=table_data, colLabels=column_labels, cellLoc='center', loc='center')

        # 테이블 스타일링 (선택 사항)
        table.auto_set_font_size(False)
        table.set_fontsize(7)
        table.scale(1.1, 5)  # 테이블 크기 조정

        # PDF에 저장
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)  # 플롯 닫기

print(f'PDF 파일이 {pdf_file_path}에 저장되었습니다.')

PDF 파일이 output.pdf에 저장되었습니다.
